In [6]:
import sys
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import cv2
import pickle 
from sklearn.metrics import accuracy_score
import shutil
# Keras Libraries
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from tensorflow.keras.utils import load_img, img_to_array
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from keras.utils import  load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report, roc_auc_score
import keras.backend
import os
import random
import time
import datetime
from tensorflow.python.framework import ops
import inspect
import gc
import tensorflow_hub as hub

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
batch_size =16


In [8]:
from sklearn.utils import class_weight as cw

def get_weight(t):
    class_weight_current =  cw.compute_class_weight('balanced', classes=np.unique(t), y=t)
    return class_weight_current


In [9]:
model_name='vgg16'
base_model = tf.keras.applications.VGG16(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(300, 600,3),
                     
                     )
 
#base_model.trainable=True
for layer in base_model.layers:
        layer.trainable=False
# For freezing the layer we make use of layer.trainable = False
# means that its internal state will not change during training.
# model's trainable weights will not be updated during fit(),
# and also its state updates will not run.
 
model = tf.keras.Sequential([
        base_model , 
        
        #tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        #tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        #tf.keras.layers.Dense(256, activation='relu'),
        
        #tf.keras.layers.Dense(128, activation='relu'),

        #tf.keras.layers.Dense(60, activation='relu'),
        
        
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 9, 18, 512)        14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 1024)              525312    
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 4)                 2052      
                                                      

In [12]:
accuracies = []
stats=[]

In [ ]:
epochs=50

# Define the path to the 5 folds
folders = ["/content/gdrive/MyDrive/Dataset/data_fold4","/content/gdrive/MyDrive/Dataset/data_fold3", "/content/gdrive/MyDrive/Dataset/data_fold2","/content/gdrive/MyDrive/Dataset/data_fold1","/content/gdrive/MyDrive/Dataset/data_fold0"]

# Initialize a list to store the accuracy scores


# Iterate over the 5 folds
for fold in folders:

    fold_name = fold.split("/")[-1]
    early = tf.keras.callbacks.EarlyStopping( patience=25,
                                          min_delta=0.001,
                                              restore_best_weights=True)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/gdrive/MyDrive/Dataset/output/checkpoints/Modelmax+min__{}_{}.h5".format(model_name,fold_name),
      save_best_only=True,
       restore_best_weights=True,
        verbose=1
    )
    reduce1=tf.keras.callbacks.ReduceLROnPlateau(factor=0.1,  # reduce the learning rate by a factor of 0.1
        patience=10,  
        
        verbose=1)
    callbacks=[early,checkpoint,reduce1]



    train_path = fold + r"/train"
   
    test_path = fold + r"/test"

    # Define the data generator
    datagen = ImageDataGenerator( rescale=1.0/255.0,
                                   shear_range = 0.02,
                                   zoom_range = 0.02, rotation_range=10,
                                  fill_mode='nearest',
                                  width_shift_range=0.1,
                                  height_shift_range=0.1
                                 )

    # Define the generator that will read images from the fold directory and their labels
    
    train_generator = datagen.flow_from_directory(directory = train_path, 
                                                        color_mode="rgb",
                                                        target_size = (300, 600), # image height , image width
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        seed=42)
    
    test_generator = datagen.flow_from_directory(directory = test_path, 
                                                    color_mode="rgb",
                                                        target_size = (300, 600), # image height , image width
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        seed=42)

    test_generator.reset()
    train_classes = train_generator.classes

    class_weights =  get_weight(train_classes)
    Class_weight = dict(zip(np.unique(train_classes), class_weights))


    model.compile(optimizer=keras.optimizers.Adam(learning_rate=.0001),loss='categorical_crossentropy',metrics=['accuracy'])

    STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID = test_generator.n//test_generator.batch_size
    # Train the model on the images in the fold
    model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs,
                    callbacks=callbacks,
              class_weight=Class_weight)
    



    model.save("/content/gdrive/MyDrive/Dataset/output/tmp/models/Modelmax+min__{}_{}.h5".format(model_name,fold_name))



    train_loss = model.history.history['loss']
    val_loss = model.history.history['val_loss']
    #print(list(cnn_model.history.keys()))
    ## Define train & validation AUC
    train_auc_name = list(model.history.history.keys())[1]
    val_auc_name = list(model.history.history.keys())[3]
    train_auc = model.history.history[train_auc_name]
    val_auc = model.history.history[val_auc_name]

    # Make predictions on the test data
    y_pred = model.predict(test_generator, steps = len(test_generator))
    y_pred = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes

    fig = plt.figure(figsize=(13, 10))
    ## PLOT 1: TRAIN VS. VALIDATION LOSS 
    plt.subplot(2,2,1)
    plt.title("Training vs. Validation Loss")
    plt.plot(train_loss, label='training loss')
    plt.plot(val_loss, label='validation loss')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()

    ## PLOT 2: TRAIN VS. VALIDATION AUC
    plt.subplot(2,2,2)
    plt.title("Training vs. Validation Acc Score")
    plt.plot(train_auc, label='training acc')
    plt.plot(val_auc, label='validation acc')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()
    
    ## PLOT 3: CONFUSION MATRIX
    plt.subplot(2,2,3)
    # Set up the labels for in the confusion matrix
    cm = confusion_matrix(y_true, y_pred,labels=[0,1,2,3])
    names = ['True Negatives', 'False Positives', 'False Negatives', 'True Positives']
    counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
    #print(counts)
    percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
    labels = [[f'{names[i]}\n{percentages[i]}' for i in range(len(names))] for j in range(len(cm))]
    #print(labels)
    #labels = np.asarray(labels).reshape(2,2)
    ticklabels = ['socre0', 'socre1','socre2','socre3']
    #print(cm)
    #print(labels)
    # Create confusion matrix as heatmap
    sns.set(font_scale = 1.4)
    ax = sns.heatmap(cm, annot=True, fmt='', cmap='Oranges', xticklabels=ticklabels, yticklabels=ticklabels )
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.title("Confusion Matrix") #plt.title("Confusion Matrix\n", fontsize=10)
    plt.xlabel("Predicted", size=14)
    plt.ylabel("Actual", size=14)
    if os.path.exists('/content/gdrive/MyDrive/Dataset/output/figures/Modelmax+min__{}_{}.jpg'.format(model_name,fold_name)):
      os.remove('/content/gdrive/MyDrive/Dataset/output/figures/Modelmax+min__{}_{}.jpg'.format(model_name,fold_name))
    plt.savefig('/content/gdrive/MyDrive/Dataset/output/figures/Modelmax+min__{}_{}.jpg'.format(model_name,fold_name))
    plt.close()
    # Evaluate the model's performance
    accuracy = accuracy_score(test_generator.classes, y_pred)
    accuracies.append(accuracy)
    df = pd.DataFrame(accuracies)
    df.to_csv('/content/gdrive/MyDrive/Dataset/output/accuracy_{}_{}.csv'.format(model_name,fold_name), index=False)
    accuracy = np.trace(cm) / np.sum(cm)
    precision = np.diag(cm) / np.sum(cm, axis = 0)
    precision = np.mean(precision)  
    recall = np.diag(cm) / np.sum(cm, axis = 1)
    recall = np.mean(recall)
    f1 = 2 * (precision * recall) / (precision + recall)
    TN = np.sum(np.diag(cm)) - np.sum(cm)
    FP = np.sum(cm, axis=0) - np.diag(cm)
    specificity = TN / (TN+FP) # % negative out of all supposed to be negatives
    specificity = np.mean(specificity)
    
    stats_summary = '[Summary Statistics]\nAccuracy = {:.2%} | Precision = {:.2%} | Recall = {:.2%} | Specificity = {:.2%} | F1 Score = {:.2%}'.format(accuracy, precision, recall, specificity, f1)
    stats.append(stats_summary)
# Print the mean accuracy
print("Mean Accuracy: {:.2f}%".format(np.mean(accuracies) * 100))

In [ ]:
accuracies

In [ ]:
stats

In [ ]:

df = pd.DataFrame(accuracies)
df.to_csv('/content/gdrive/MyDrive/Dataset/output/{}_accuracies.csv'.format(model_name), index=False)
df = pd.DataFrame(stats)
df.to_csv('/content/gdrive/MyDrive/Dataset/output/{}_stats.csv'.format(model_name), index=False)

# fine tuning


In [ ]:
accuracies=[]

In [ ]:
epochs=50

# Define the path to the 5 folds
folders = ["/content/gdrive/MyDrive/Dataset/data_fold4","/content/gdrive/MyDrive/Dataset/data_fold3", "/content/gdrive/MyDrive/Dataset/data_fold2","/content/gdrive/MyDrive/Dataset/data_fold1","/content/gdrive/MyDrive/Dataset/data_fold0"]

# Initialize a list to store the accuracy scores


# Iterate over the 5 folds
for fold in folders:

    fold_name = fold.split("/")[-1]
    fold_name1 = fold.split("_")[-1]




    model= tf.keras.models.load_model("/content/gdrive/MyDrive/Dataset/output/models/Modelmax+min__mobilnetv2_{}.h5".format(fold_name))
    for layer in model.layers:
        layer.trainable=True
    model.summary()
    model.compile(tf.keras.optimizers.RMSprop(learning_rate=.000001),loss='categorical_crossentropy',metrics=['accuracy'])






    early = tf.keras.callbacks.EarlyStopping( patience=25,
                                          min_delta=0.001,
                                              restore_best_weights=True)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/gdrive/MyDrive/Dataset/output/checkpoints/Modelmax+min__mobilnetv2_finetuned_{}.h5".format(fold_name),
      save_best_only=True,
       restore_best_weights=True,
        verbose=1
    )
    reduce1=tf.keras.callbacks.ReduceLROnPlateau(factor=0.1,  # reduce the learning rate by a factor of 0.1
        patience=10,  
        
        verbose=1)
    callbacks=[early,checkpoint,reduce1]





    train_path = fold + r"/train"
   
    test_path = fold + r"/test"

    # Define the data generator
    datagen = ImageDataGenerator( rescale=1.0/255.0,
                                    rotation_range=5,
                                 )

    # Define the generator that will read images from the fold directory and their labels
    
    train_generator = datagen.flow_from_directory(directory = train_path, 
                                                        color_mode="rgb",
                                                        target_size = (300, 600), # image height , image width
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        seed=42)
    
    test_generator = datagen.flow_from_directory(directory = test_path, 
                                                    color_mode="rgb",
                                                        target_size = (300, 600), # image height , image width
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        seed=42)

    test_generator.reset()
   
    

    STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID = test_generator.n//test_generator.batch_size
    # Train the model on the images in the fold
    model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs,
                    callbacks=callbacks,
             )
    



    model.save("/content/gdrive/MyDrive/Dataset/output/models/Modelmax+min__mobilnetv2_finetuned_{}.h5".format(fold_name))



    train_loss = model.history.history['loss']
    val_loss = model.history.history['val_loss']
    #print(list(cnn_model.history.keys()))
    ## Define train & validation AUC
    train_auc_name = list(model.history.history.keys())[1]
    val_auc_name = list(model.history.history.keys())[3]
    train_auc = model.history.history[train_auc_name]
    val_auc = model.history.history[val_auc_name]

    # Make predictions on the test data
    y_pred = model.predict(test_generator, steps = len(test_generator))
    y_pred = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes

    fig = plt.figure(figsize=(13, 10))
    ## PLOT 1: TRAIN VS. VALIDATION LOSS 
    plt.subplot(2,2,1)
    plt.title("Training vs. Validation Loss")
    plt.plot(train_loss, label='training loss')
    plt.plot(val_loss, label='validation loss')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()

    ## PLOT 2: TRAIN VS. VALIDATION AUC
    plt.subplot(2,2,2)
    plt.title("Training vs. Validation Acc Score")
    plt.plot(train_auc, label='training acc')
    plt.plot(val_auc, label='validation acc')
    plt.xlabel("Number of Epochs", size=14)
    plt.legend()
    
    ## PLOT 3: CONFUSION MATRIX
    plt.subplot(2,2,3)
    # Set up the labels for in the confusion matrix
    cm = confusion_matrix(y_true, y_pred,labels=[0,1,2,3])
    names = ['True Negatives', 'False Positives', 'False Negatives', 'True Positives']
    counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
    #print(counts)
    percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
    labels = [[f'{names[i]}\n{percentages[i]}' for i in range(len(names))] for j in range(len(cm))]
    #print(labels)
    #labels = np.asarray(labels).reshape(2,2)
    ticklabels = ['socre0', 'socre1','socre2','socre3']
    #print(cm)
    #print(labels)
    # Create confusion matrix as heatmap
    sns.set(font_scale = 1.4)
    ax = sns.heatmap(cm, annot=True, fmt='', cmap='Oranges', xticklabels=ticklabels, yticklabels=ticklabels )
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.title("Confusion Matrix") #plt.title("Confusion Matrix\n", fontsize=10)
    plt.xlabel("Predicted", size=14)
    plt.ylabel("Actual", size=14)
    if os.path.exists('/content/gdrive/MyDrive/Dataset/output/figures/Modelmax+min__mobilnetv2_finetuned_{}.jpg'.format(fold_name)):
      os.remove('/content/gdrive/MyDrive/Dataset/output/figures/Modelmax+min__mobilnetv2_finetuned_{}.jpg'.format(fold_name))
    plt.savefig('/content/gdrive/MyDrive/Dataset/output/figures/Modelmax+min__mobilnetv2_finetuned_{}.jpg'.format(fold_name))
    plt.close()
    # Evaluate the model's performance
    accuracy = accuracy_score(test_generator.classes, y_pred)
    accuracies.append(accuracy)
    df = pd.DataFrame(accuracies)
    df.to_csv('/content/gdrive/MyDrive/Dataset/output/accuracy_mobilnetv2_finetuned_{}.csv'.format(fold_name), index=False)
    accuracy = np.trace(cm) / np.sum(cm)
    precision = np.diag(cm) / np.sum(cm, axis = 0)
    precision = np.mean(precision)  
    recall = np.diag(cm) / np.sum(cm, axis = 1)
    recall = np.mean(recall)
    f1 = 2 * (precision * recall) / (precision + recall)
    TN = np.sum(np.diag(cm)) - np.sum(cm)
    FP = np.sum(cm, axis=0) - np.diag(cm)
    specificity = TN / (TN+FP) # % negative out of all supposed to be negatives
    specificity = np.mean(specificity)
    
    stats_summary = '[Summary Statistics]\nAccuracy = {:.2%} | Precision = {:.2%} | Recall = {:.2%} | Specificity = {:.2%} | F1 Score = {:.2%}'.format(accuracy, precision, recall, specificity, f1)
    stats.append(stats_summary)
# Print the mean accuracy
print("Mean Accuracy: {:.2f}%".format(np.mean(accuracies) * 100))

In [ ]:
df = pd.DataFrame(accuracies)
df.to_csv('/content/gdrive/MyDrive/Dataset/output/mobilnetv2_finetuned_accuracies.csv', index=False)
df = pd.DataFrame(stats)
df.to_csv('/content/gdrive/MyDrive/Dataset/output/mobilnetv2_finetuned_stats.csv', index=False)